In [35]:
import json
import pickle
import pyodbc
import pandas as pd
from tqdm import tqdm

from constants import LFM_COUNTRY_CODE 

In [28]:
class Database:
    def __init__(self, env = 'staging'):
        with open('../config/database.json') as file:
            database_info = json.load(file)
        self.conn_common = self._database_conn(database_info[env], 'common')
        self.conn_dmp = self._database_conn(database_info[env], 'dmp')
        
    def _database_conn(self, database_info, database):
        return 'Driver={};\
                Server={};\
                Database={};\
                Trusted_Connection=yes'.format(
            database_info['driver'], database_info['server'], 
            database_info['database'][database]
        )
    
    def test_conn(self):
        conn = pyodbc.connect(self.conn_dmp)
        query = """SELECT
                        TOP (10) *
                   FROM
                        dbo.AgeRangeByContentFormatId  
                """
        cursor = conn.cursor()
        cursor.execute(query)
        if cursor:
            print('Connection successful')
        else:
            print('Connection UNsuccessful')
        cursor.close()
        
    def write_gender(self, data):
        conn = pyodbc.connect(self.conn_dmp)
        query = """INSERT INTO 
                    dbo.GenderByContentFormatId (ContentFormatId, CountryCode, IsMale, Percentage, ContentFormatSourceId, GeoItemId)
                   VALUES 
                    ({}, '{}', '{}', {}, 3, Null)
                """.format(data['ContentFormatId'], data['CountryCode'], data['IsMale'], data['Percentage'])
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        cursor.close()
        
    def write_age(self, data):
        conn = pyodbc.connect(self.conn_dmp)
        query = """INSERT INTO 
                    dbo.AgeRangeByContentFormatId (ContentFormatId, CountryCode, MinAge, MaxAge, Percentage, ContentFormatSourceId, GeoItemId)
                   VALUES 
                    ({}, '{}', {}, {}, {}, 3, Null)
                """.format(data['ContentFormatId'], data['CountryCode'], data['MinAge'], data['MaxAge'], data['Percentage'])
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        cursor.close()

In [40]:
db = Database(env = 'prod')

'''
data_gender = {
    'ContentFormatId': 342,
    'CountryCode': 'US',
    'IsMale': False,
    'Percentage': 12.3
}
'''

def write_gender(code):
    df = pd.read_csv('../data/gender/gender_{}.csv'.format(code))
    data = df.to_dict('records')
    for row in tqdm(data):
        if (code != 'LFM'):
#             db.write_gender(row)
            print(row)
        elif (row['CountryCode'] in LFM_COUNTRY_CODE):
#             db.write_gender(row)
            print(row)
    print('Done writing gender data for', code)

'''
data_age = {
    'ContentFormatId': 342,
    'CountryCode': 'US',
    'MinAge': 'Null',
    'MaxAge': 24,
    'Percentage': 12.3
}
'''

def write_age(code):
    df = pd.read_csv('../data/age/age_{}.csv'.format(code))
    data = df.to_dict('records')
    for row in tqdm(data):
        if (row['MinAge'] == 0): row['MinAge'] = 'Null'
        if (row['MaxAge'] == 100): row['MaxAge'] = 'Null'
        if (code != 'LFM'):
#             db.write_age(row)
            print(row)
        elif (row['CountryCode'] in LFM_COUNTRY_CODE):
#             db.write_age(row)
            print(row)
    print('Done writing age data for', code)

['AR', 'AT', 'BE', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CU', 'CY', 'DE', 'DK', 'DO', 'EC', 'ES', 'FI', 'FR', 'GR', 'GT', 'HN', 'HU', 'IE', 'IS', 'IT', 'JM', 'LU', 'MX', 'NO', 'PA', 'PE', 'PH', 'PR', 'PT', 'PY', 'SE', 'SV', 'UY', 'VE']


In [49]:
write_age('NL')
write_age('UK')
write_age('LFM')

100%|██████████| 192/192 [00:00<00:00, 191193.34it/s]


{'Unnamed: 0': 0, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 7.2}
{'Unnamed: 0': 1, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 11.08}
{'Unnamed: 0': 2, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 35, 'MaxAge': 44, 'Percentage': 9.66}
{'Unnamed: 0': 3, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 45, 'MaxAge': 54, 'Percentage': 13.05}
{'Unnamed: 0': 4, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 55, 'MaxAge': 64, 'Percentage': 23.17}
{'Unnamed: 0': 5, 'ContentFormatId': 346, 'CountryCode': 'NL', 'MinAge': 65, 'MaxAge': 'Null', 'Percentage': 35.84}
{'Unnamed: 0': 6, 'ContentFormatId': 367, 'CountryCode': 'NL', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 9.85}
{'Unnamed: 0': 7, 'ContentFormatId': 367, 'CountryCode': 'NL', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 12.24}
{'Unnamed: 0': 8, 'ContentFormatId': 367, 'CountryCode': 'NL', 'MinAge': 35, 'MaxAge': 44, 'Perc

100%|██████████| 102/102 [00:00<00:00, 207276.65it/s]


{'Unnamed: 0': 0, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 11.46}
{'Unnamed: 0': 1, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 13.1}
{'Unnamed: 0': 2, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 35, 'MaxAge': 44, 'Percentage': 18.01}
{'Unnamed: 0': 3, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 45, 'MaxAge': 54, 'Percentage': 27.13}
{'Unnamed: 0': 4, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 55, 'MaxAge': 64, 'Percentage': 19.23}
{'Unnamed: 0': 5, 'ContentFormatId': 342, 'CountryCode': 'UK', 'MinAge': 65, 'MaxAge': 'Null', 'Percentage': 11.07}
{'Unnamed: 0': 6, 'ContentFormatId': 365, 'CountryCode': 'UK', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 18.04}
{'Unnamed: 0': 7, 'ContentFormatId': 365, 'CountryCode': 'UK', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 21.1}
{'Unnamed: 0': 8, 'ContentFormatId': 365, 'CountryCode': 'UK', 'MinAge': 35, 'MaxAge': 44, 'Pe

100%|██████████| 6768/6768 [00:00<00:00, 451024.79it/s]

{'Unnamed: 0': 0, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 45.13}
{'Unnamed: 0': 1, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 38.78}
{'Unnamed: 0': 2, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 35, 'MaxAge': 44, 'Percentage': 8.52}
{'Unnamed: 0': 3, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 45, 'MaxAge': 54, 'Percentage': 6.17}
{'Unnamed: 0': 4, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 55, 'MaxAge': 64, 'Percentage': 1.13}
{'Unnamed: 0': 5, 'ContentFormatId': 344, 'CountryCode': 'DE', 'MinAge': 65, 'MaxAge': 'Null', 'Percentage': 0.26}
{'Unnamed: 0': 18, 'ContentFormatId': 346, 'CountryCode': 'AR', 'MinAge': 'Null', 'MaxAge': 24, 'Percentage': 31.07}
{'Unnamed: 0': 19, 'ContentFormatId': 346, 'CountryCode': 'AR', 'MinAge': 25, 'MaxAge': 34, 'Percentage': 44.3}
{'Unnamed: 0': 20, 'ContentFormatId': 346, 'CountryCode': 'AR', 'MinAge': 35, 'MaxAge': 44, 'Pe

In [50]:
write_gender('NL')
write_gender('UK')
write_gender('LFM')

100%|██████████| 64/64 [00:00<?, ?it/s]


{'Unnamed: 0': 0, 'ContentFormatId': 346, 'CountryCode': 'NL', 'IsMale': False, 'Percentage': 49.93}
{'Unnamed: 0': 1, 'ContentFormatId': 346, 'CountryCode': 'NL', 'IsMale': True, 'Percentage': 50.07}
{'Unnamed: 0': 2, 'ContentFormatId': 367, 'CountryCode': 'NL', 'IsMale': False, 'Percentage': 40.57}
{'Unnamed: 0': 3, 'ContentFormatId': 367, 'CountryCode': 'NL', 'IsMale': True, 'Percentage': 59.43}
{'Unnamed: 0': 4, 'ContentFormatId': 208, 'CountryCode': 'NL', 'IsMale': False, 'Percentage': 46.69}
{'Unnamed: 0': 5, 'ContentFormatId': 208, 'CountryCode': 'NL', 'IsMale': True, 'Percentage': 53.31}
{'Unnamed: 0': 6, 'ContentFormatId': 353, 'CountryCode': 'NL', 'IsMale': False, 'Percentage': 50.11}
{'Unnamed: 0': 7, 'ContentFormatId': 353, 'CountryCode': 'NL', 'IsMale': True, 'Percentage': 49.89}
{'Unnamed: 0': 8, 'ContentFormatId': 363, 'CountryCode': 'NL', 'IsMale': False, 'Percentage': 39.75}
{'Unnamed: 0': 9, 'ContentFormatId': 363, 'CountryCode': 'NL', 'IsMale': True, 'Percentage': 60

100%|██████████| 34/34 [00:00<?, ?it/s]


{'Unnamed: 0': 0, 'ContentFormatId': 342, 'CountryCode': 'UK', 'IsMale': False, 'Percentage': 53.17}
{'Unnamed: 0': 1, 'ContentFormatId': 342, 'CountryCode': 'UK', 'IsMale': True, 'Percentage': 46.83}
{'Unnamed: 0': 2, 'ContentFormatId': 365, 'CountryCode': 'UK', 'IsMale': False, 'Percentage': 43.24}
{'Unnamed: 0': 3, 'ContentFormatId': 365, 'CountryCode': 'UK', 'IsMale': True, 'Percentage': 56.76}
{'Unnamed: 0': 4, 'ContentFormatId': 361, 'CountryCode': 'UK', 'IsMale': False, 'Percentage': 45.39}
{'Unnamed: 0': 5, 'ContentFormatId': 361, 'CountryCode': 'UK', 'IsMale': True, 'Percentage': 54.61}
{'Unnamed: 0': 6, 'ContentFormatId': 345, 'CountryCode': 'UK', 'IsMale': False, 'Percentage': 43.96}
{'Unnamed: 0': 7, 'ContentFormatId': 345, 'CountryCode': 'UK', 'IsMale': True, 'Percentage': 56.04}
{'Unnamed: 0': 8, 'ContentFormatId': 346, 'CountryCode': 'UK', 'IsMale': False, 'Percentage': 44.370000000000005}
{'Unnamed: 0': 9, 'ContentFormatId': 346, 'CountryCode': 'UK', 'IsMale': True, 'Pe

100%|██████████| 2256/2256 [00:00<00:00, 451383.38it/s]

{'Unnamed: 0': 0, 'ContentFormatId': 344, 'CountryCode': 'DE', 'IsMale': True, 'Percentage': 74.96}
{'Unnamed: 0': 1, 'ContentFormatId': 344, 'CountryCode': 'DE', 'IsMale': False, 'Percentage': 25.04}
{'Unnamed: 0': 6, 'ContentFormatId': 346, 'CountryCode': 'AR', 'IsMale': True, 'Percentage': 85.41}
{'Unnamed: 0': 7, 'ContentFormatId': 346, 'CountryCode': 'AR', 'IsMale': False, 'Percentage': 14.59}
{'Unnamed: 0': 8, 'ContentFormatId': 346, 'CountryCode': 'AT', 'IsMale': True, 'Percentage': 84.55}
{'Unnamed: 0': 9, 'ContentFormatId': 346, 'CountryCode': 'AT', 'IsMale': False, 'Percentage': 15.45}
{'Unnamed: 0': 12, 'ContentFormatId': 346, 'CountryCode': 'BE', 'IsMale': True, 'Percentage': 82.62}
{'Unnamed: 0': 13, 'ContentFormatId': 346, 'CountryCode': 'BE', 'IsMale': False, 'Percentage': 17.38}
{'Unnamed: 0': 16, 'ContentFormatId': 346, 'CountryCode': 'BR', 'IsMale': True, 'Percentage': 71.34}
{'Unnamed: 0': 17, 'ContentFormatId': 346, 'CountryCode': 'BR', 'IsMale': False, 'Percentage'